In [2]:
import kipoi
model_name = "DeepSEA/variantEffects"
# get the model
model = kipoi.get_model(model_name)
# get the dataloader factory
#Dataloader = kipoi.get_dataloader_factory(model_name)

Using downloaded and verified file: /home/isshamie/.kipoi/models/DeepSEA/variantEffects/downloaded/model_files/weights/35956ab9c28960b5a3693f470fe980c1


In [3]:
# INDIR = "data/processed/mttrace/TcellDupi_may17_2021/MTblacklist/merged/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/filter_mgatk/vireoIn/clones"
# OUTDIR = "data/processed/mttrace/TcellDupi_may17_2021/MTblacklist/merged/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/filter_mgatk/vireoIn/clones/n_clones_20"
# N_DONORS=2
# vars_to_plot=10
# sample_names="pre,post"
# n_clones=20
# var_thresh = 0.001
# vcf_f=None

# The input vcf path
raw_vcf = "/data2/mito_lineage/data/processed/mttrace/TcellDupi_may17_2021/MTblacklist/merged/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/filter_mgatk/vireoIn/cellSNP.base.vcf"
mt_fasta = "/data2/mito_lineage/data/external/GRCh38_MT_blacklist/chrM.fasta" # "example_data/hg19_chr22.fa"

In [4]:
vcf_path = raw_vcf.replace(".vcf", ".fmt.vcf")

## Create proper formatted vcf file

In [5]:
# import pandas as pd
# vcf = pd.read_csv(raw_vcf, sep='\t')
# vcf["QUAL"] = "."
# vcf["FILTER"] = "."
# vcf["INFO"] = "."
# vcf["ID"] = "."
# vcf["REF"] = vcf["REF"].apply(lambda x: x[-1])
# vcf = vcf[["#CHROM" ,"POS","ID","REF","ALT","QUAL","FILTER","INFO"]]
# vcf.to_csv(vcf_path, sep='\t', index=False)

In [6]:
# The output vcf path, based on the input file name    
out_vcf_fpath = vcf_path[:-4] + "%s.vcf"%model_name.replace("/", "_")
# The datalaoder keyword arguments
# The datalaoder keyword arguments
dataloader_arguments = {"fasta_file": "/data2/mito_lineage/data/external/GRCh38_MT_blacklist/fasta/genome.fa",
                       "gtf_file":"/data/isshamie/mito_lineage/data/external/GRCh38_MT_blacklist/genes/genes.gtf",}



In [7]:
# Now actually run the effect prediction using the logit difference:
import kipoi_veff.snv_predict as sp
sp.score_variants(model = model_name,
              dl_args = dataloader_arguments,
                  scores = ["logit"],
                  input_vcf = vcf_path,
                  output_vcf = out_vcf_fpath)

Using downloaded and verified file: /home/isshamie/.kipoi/models/DeepSEA/variantEffects/downloaded/model_files/weights/35956ab9c28960b5a3693f470fe980c1
INFO [kipoi_veff.snv_predict] Using variant-centered sequence generation.


TypeError: __init__() got an unexpected keyword argument 'gtf_file'

In [12]:
kipoi.get_dataloader_descr(dataloader_arguments)


TypeError: expected str, bytes or os.PathLike object, not dict

In [11]:
import varcode


In [14]:

# Load TCGA MAF containing variants from their
variants = varcode.load_vcf(vcf_path)

print(variants)
### <VariantCollection from 'tcga-ovarian-cancer-variants.maf' with 6428 elements>
###  -- Variant(contig=1, start=69538, ref=G, alt=A, genome=GRCh37)
###  -- Variant(contig=1, start=881892, ref=T, alt=G, genome=GRCh37)
###  -- Variant(contig=1, start=3389714, ref=G, alt=A, genome=GRCh37)
###  -- Variant(contig=1, start=3624325, ref=G, alt=T, genome=GRCh37)
###  ...

# you can index into a VariantCollection and get back a Variant object
variant = variants[0]

# groupby_gene_name returns a dictionary whose keys are gene names
# and whose values are themselves VariantCollections
gene_groups = variants.groupby_gene_name()

# get variants which affect the TP53 gene
TP53_variants = gene_groups["TP53"]

# predict protein coding effect of every TP53 variant on
# each transcript of the TP53 gene
TP53_effects = TP53_variants.effects()

print(TP53_effects)
### <EffectCollection with 789 elements>
### -- PrematureStop(variant=chr17 g.7574003G>A, transcript_name=TP53-001, transcript_id=ENST00000269305, effect_description=p.R342*)
### -- ThreePrimeUTR(variant=chr17 g.7574003G>A, transcript_name=TP53-005, transcript_id=ENST00000420246)
### -- PrematureStop(variant=chr17 g.7574003G>A, transcript_name=TP53-002, transcript_id=ENST00000445888, effect_description=p.R342*)
### -- FrameShift(variant=chr17 g.7574030_7574030delG, transcript_name=TP53-001, transcript_id=ENST00000269305, effect_description=p.R333fs)
### ...

premature_stop_effect = TP53_effects[0]

print(str(premature_stop_effect.mutant_protein_sequence))
### 'MEEPQSDPSVEPPLSQETFSDLWKLLPENNVLSPLPSQAMDDLMLSPDDIEQWFTEDPGPDEAPRMPEAAPPVAPAPAAPTPAAPAPAPSWPLSSSVPSQKTYQGSYGFRLGFLHSGTAKSVTCTYSPALNKMFCQLAKTCPVQLWVDSTPPPGTRVRAMAIYKQSQHMTEVVRRCPHHERCSDSDGLAPPQHLIRVEGNLRVEYLDDRNTFRHSVVVPYEPPEVGSDCTTIHYNYMCNSSCMGGMNRRPILTIITLEDSSGNLLGRNSFEVRVCACPGRDRRTEEENLRKKGEPHHELPPGSTKRALPNNTSSSPQPKKKPLDGEYFTLQIRGRERFEMF'

print(premature_stop_effect.aa_mutation_start_offset)
### 341

print(premature_stop_effect.transcript)
### Transcript(id=ENST00000269305, name=TP53-001, gene_name=TP53, biotype=protein_coding, location=17:7571720-7590856)

print(premature_stop_effect.gene.name)
### 'TP53'

<VariantCollection from '/data2/mito_lineage/data/processed/mttrace/TcellDupi_may17_2021/MTblacklist/merged/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/filter_mgatk/vireoIn/cellSNP.base.fmt.vcf' with 1433 elements>
  -- Variant(contig='chrM', start=49, ref='A', alt='G', reference_name='GRCh38')
  -- Variant(contig='chrM', start=49, ref='A', alt='C', reference_name='GRCh38')
  -- Variant(contig='chrM', start=49, ref='A', alt='T', reference_name='GRCh38')
  -- Variant(contig='chrM', start=57, ref='T', alt='C', reference_name='GRCh38')
  -- Variant(contig='chrM', start=57, ref='T', alt='G', reference_name='GRCh38')
  -- Variant(contig='chrM', start=57, ref='T', alt='A', reference_name='GRCh38')
  -- Variant(contig='chrM', start=58, ref='T', alt='G', reference_name='GRCh38')
  -- Variant(contig='chrM', start=58, ref='T', alt='C', reference_name='GRCh38')
  -- Variant(contig='chrM', start=58, ref='T', alt='A', reference_name='GRCh38')
  -- Variant(contig='chrM', start=64, re

/data/isshamie/software/anaconda3/envs/mito_trace/lib/python3.6/site-packages/varcode/reference.py:285: UserWarning: Reference could not be matched against filename (file://data2/mito_lineage/data/external/GRCh38_MT_blacklist/fasta/genome.fa); using best match against full path (GRCh38).
  reference_name_or_path, match))


ValueError: Invalid contig name 'chrM' for reference 'GRCh38'

In [6]:
import kipoi_veff.snv_predict as sp
sp.score_variants(model = model_name,
                  dl_args = dataloader_arguments,
                  input_vcf = vcf_path,
                  output_vcf = out_vcf_fpath)

Using downloaded and verified file: /home/isshamie/.kipoi/models/DeepSEA/variantEffects/downloaded/model_files/weights/35956ab9c28960b5a3693f470fe980c1
INFO [kipoi_veff.snv_predict] Using variant-centered sequence generation.


EmptyDataError: No columns to parse from file

In [ ]:
vcf_path